# Задание 2
## СУБД в высоконагруженных проектах

### Введение
Используется Django Rest Framework (т.е. с базой общается Django ORM), на раздачу статики настроен Nginx (порт 80), на более сложные запросы gunicorn (4 воркера). База mysql, движок InnoDB.

Запрос на чтение происходит при обращении к http://localhost/books/, выдаются те книги в названии которых есть 2 буквы, в каждом запросе эти 2 буквы генерируются случайным образом. В ORM это выглядит так: 
    
**`queryset = Book.objects.filter(title__startswith=generate())`**

In [ ]:
# application/urls.py 
from books.models import Book
from django.conf.urls import url, include
from django.contrib import admin
from rest_framework import routers, serializers, viewsets
import random


class BookSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Book
        fields = ('id', 'title', 'content')


def generate():
    symbols = 'ABCDEFGHIGKLMNOPQRSTUVWXYZabcdefghigklmnopqrstuvwxyz'
    newstr = [symbols[random.randint(0, 51)], symbols[random.randint(0, 51)]]
    return ''.join(newstr)


# ViewSets define the view behavior.
class BookViewSet(viewsets.ModelViewSet):
    # fitler objects with LIKE 'ab%', ab - two random letters
    queryset = Book.objects.filter(title__startswith=generate())
    serializer_class = BookSerializer


# Routers provide an easy way of automatically determining the URL conf.
router = routers.DefaultRouter()
router.register(r'books', BookViewSet)

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    url(r'^', include(router.urls)),
]

### Без индекса на поле title
Проведём тесты при различных профилях нагрузки:
* 95% чтение, 5% запись
* 50% чтение, 50% запись
* 5% чтение, 95% запись

Тестирование будем провдить с помощью программы [JMeter](https://jmeter.apache.org/).

#### 95% чтение, 5% запись
Заранее измеряем max throuput. В моём случае он оказался 1180 запросов в минуту. Создадим 2 группы пользователей: 95 на чтение и 5 на запись. Поставим для каждой группы пользователей Constant Throuput Timer и укажем у читателей потолок в 1121.0, а у тех, кто пишет 59.0 (в принципе это необязательно, но тогда отношение читающих к пишущим ближе к 95/5, чем если не указывать этот таймер). Делаем динамическую нагрузку, все пользователи включаются в работу в течение 30 секунд. Тест проводим в течение 2-ух минут

![Без индекса, 95% чтение, 5% запись](https://lh3.googleusercontent.com/vogXQiX7Micyggj6NzPbClMMulSzZLp9xLiEYpBqof7RLaKa7u1RSY6PliD5Tu_f4IW8L3Ra11lGkso=w1366-h646 "Без индекса, 95% чтение, 5% запись")
![Без индекса, 95% чтение, 5% запись, аггрегированные данные](https://lh6.googleusercontent.com/2oW4G8zGKrVykBpjaYK76U3l1Pe2-u4LSOj-wuvVBC8v09umuVtFtWkC4eOAYnq4JI_i7h8hG0-B6cg=w1366-h646 "Без индекса, 95% чтение, 5% запись, аггрегированные данные")

**Средний Latency: 5860**

#### 50% чтение, 50% запись
Не забываем поменять Constant Throuput Timer-ы и количество пользователей в каждой группе. Получаем

![Без индекса, 50% чтение, 50% запись](https://lh6.googleusercontent.com/q-J4mC7t_oSDEAeWpVszfZK47A4ckCHM9WmDpd9AvMTcpeGjsZyJcb65FcwKFWJSlyDe0UPzwS4NB_o=w1366-h646 "Без индекса, 50% чтение, 50% запись")
![Без индекса, 50% чтение, 50% запись, аггрегированные данные](https://lh5.googleusercontent.com/6eMxNS3rNp0jZUpZboH26-xhe1ja2VxKMUYNmSEVwEYUuSQ9rTlSSdv1c6f6XKSzK4hDwAVZJxXR8eo=w1366-h646 "Без индекса, 50% чтение, 50% запись, аггрегированные данные")

**Средний Latency: 254**

#### 5% чтение, 95% запись

![Без индекса, 5% чтение, 95% запись](https://lh6.googleusercontent.com/TS54hmd3wHsMUuVAx__R-sZ94C6E-yY4Hy8qZkmUO0xihObI67n7u6IIYEc1BO71_WJCDVWbHMGOZFo=w1366-h646 "Без индекса, 5% чтение, 95% запись")
![Без индекса, 5% чтение, 95% запись, аггрегированные данные](https://lh6.googleusercontent.com/La0Nd8dhrcrA60hWPXkBQfPzbjjH2c9SKf_Wm2lw9yzhT4Yyz8X-cM7RpYKp_jCa-UfaDiPQKgKOp5c=w1366-h646 "Без индекса, 5% чтение, 95% запись, аггрегированные данные")

**Средний Latency: 120**

### С индексом на поле title
Добавим индекс на поле title:

`mysql> CREATE INDEX titleidx ON books_book (title(10));`

`mysql> EXPLAIN SELECT title FROM books_book WHERE title LIKE 'ab%';`

`Output ==>`
`
+----+-------------+------------+------------+-------+---------------+----------+---------+------+------+----------+
| id | select_type | table      | partitions | type  | possible_keys | key      | key_len | ref  | rows | filtered | 
+----+-------------+------------+------------+-------+---------------+----------+---------+------+------+----------+
|  1 | SIMPLE      | books_book | NULL       | range | titleidx      | titleidx | 32      | NULL |  110 |   100.00 | 
+----+-------------+------------+------------+-------+---------------+----------+---------+------+------+----------+
`


#### 95% чтение, 5% запись
Не забываем сделать предварительный тест, дабы "разогреть" индексы. Можно также посмотреть ещё раз max throuput, у меня он в общем-то не поменялся, так что числа в Constant Throuput Timer-ах примерно такие же, что и раньше

![Индекс на title, 95% чтение, 5% запись](https://lh6.googleusercontent.com/stuQe3UQmdJxeE2D6xlVPPmu-E74jNfyPR77rNGlRVe4YqEHX3ERHvZ3L3dBC5306w5SHzIuc_m-gEI=w1366-h646 "Индекс на title, 95% чтение, 5% запись")
![Индекс на title, 95% чтение, 5% запись, аггрегированные данные](https://lh4.googleusercontent.com/in3ZHyaOkIR7-Wi0yeX41JwrNHHbBJgdvVWY4Ku6JLTDFF3IUQydg6vqhH3uBgCBjl-NT5uLKYs2T3s=w1366-h646 "Индекс на title, 95% чтение, 5% запись, аггрегированные данные")

**Средний Latency: 5163**

Что меньше, чем без индекса на 12%. Ожидал большего

#### 50% чтение, 50% запись

![Индекс на title, 50% чтение, 50% запись](https://lh4.googleusercontent.com/FqN7GVTWAZH72zz2e-2NGcHnTRSUjZURKO1YUuGvK11fNncj3yNDKbEJoT0GENzG_lTknQrLZEsnwUY=w1366-h646 "Индекс на title, 50% чтение, 50% запись")
![Индекс на title, 50% чтение, 50% запись, аггрегированные данные](https://lh3.googleusercontent.com/wxHh7S2JuFtRA_B1_jxtOuA3Udb19m1cEYJdeTIEOPbp9TAYeM3N8Ra7kpZ-UcDV7lzVTHLfTXI_O9w=w1366-h646 "Индекс на title, 50% чтение, 50% запись, аггрегированные данные")

**Средний Latency: 280**

Результат стал хуже на 10%.

#### 5% чтение, 95% запись

![Индекс на title, 5% чтение, 95% запись](https://lh4.googleusercontent.com/CwbfmLk7-fZDGviM-F1iXbX_65Yrs1GMiA8Hi4g3PaRl_A98JgufJKN48L3VzGbpxYCsjIxy4O1KKW0=w1366-h646 "Индекс на title, 5% чтение, 95% запись")
![Индекс на title, 5% чтение, 95% запись, аггрегированные данные](https://lh4.googleusercontent.com/aXNaU3a7g6HQ9OK66CytJ_s9Hpwd9yoE-p1YFm8otam-3h7sFMEYDlCc1Dst0xEgQcFpEV4Q58hdJ1Y=w1366-h646 "Индекс на title, 5% чтение, 95% запись, аггрегированные данные")

**Средний Latency: 419**

Причём в течение теста, значения были гораздо меньше, но под конец время ответа резко возросло, возможно, в этот момент mysql сервер перестраивал индекс (на скриншоте видно этот момент). Результат гораздо хуже, чем был.

### Много плохих индексов
`mysql> CREATE INDEX bad1 ON books_book (id, title(10));`

`mysql> CREATE INDEX bad2 ON books_book (title(10), content(20));`

`mysql> CREATE INDEX bad3 ON books_book (id, content(20));`

`mysql> CREATE INDEX bad4 ON books_book (title(30), content(10));`

`mysql> CREATE INDEX bad5 ON books_book (id, title(20), content(10));`

`mysql> EXPLAIN SELECT title FROM books_book WHERE title LIKE 'ab%';`

`Output ==>`
`
+----+-------------+------------+------------+-------+--------------------+----------+---------+------+------+----------+
| id | select_type | table      | partitions | type  | possible_keys      | key      | key_len | ref  | rows | filtered | Extra       |
+----+-------------+------------+------------+-------+--------------------+----------+---------+------+------+----------+
|  1 | SIMPLE      | books_book | NULL       | range | titleidx,bad2,bad4 | titleidx | 32      | NULL |  110 |   100.00 
+----+-------------+------------+------------+-------+--------------------+----------+---------+------+------+----------+
`

#### 95% чтение, 5% запись
Также сначала "разогреем" индексы.

![Много индексов, 95% чтение, 5% запись](https://lh5.googleusercontent.com/ArvfWaxyBp2SYFon2UhDS59SSOJrMwJMiwp8N0Kkc3Ydf7n6VDbgMF2rNVX4gYlUCnmQp6v_KDixfdE=w1366-h646 "Много индексов, 95% чтение, 5% запись")
![Много индексов, 95% чтение, 5% запись, аггрегированные данные](https://lh3.googleusercontent.com/XqON5WEQYvH5NqrEDGHbR7h-rhiZAdO0gD2vrjKGKgN2ePCvCaOqr7OYoO-mIOScZRVNIn-9Ay8RfY0=w1366-h646 "Много индексов, 95% чтение, 5% запись, аггрегированные данные")

**Средний Latency: 4698**

Результат стал лучше O_o. Нуок.

#### 50% чтение, 50% запись

![Много индексов, 50% чтение, 50% запись](https://lh4.googleusercontent.com/BJohdg3HJy_TWDqVnl511aqjJIlvkISdPh3ANfbxiIDWK2OenH1cJcL7jQ_yXBx3RI4y2JQf8jcGr6A=w1366-h646 "Много индексов, 50% чтение, 50% запись")
![Много индексов, 50% чтение, 50% запись, аггрегированные данные](https://lh4.googleusercontent.com/87N0toJow4AoTARcgbptpOjOv7xYErITAiO3GfthWDpdgfhppOC-bkc_2DyVEpUokwEpfVFjWda9qjc=w1366-h646 "Много индексов, 50% чтение, 50% запись, аггрегированные данные")

**Средний Latency: 312**

#### 5% чтение, 95% запись

![Много индексов, 5% чтение, 95% запись](https://lh5.googleusercontent.com/Rp8VN0sq34vpQR5MC5uGt0MwThiv_SantVQv41Y3kfevOtXLancXGYaZ2dI1PGCgGE0ved3HOcVc-v8=w1366-h646 "Много индексов, 5% чтение, 95% запись")
![Много индексов, 5% чтение, 95% запись, аггрегированные данные](https://lh6.googleusercontent.com/HSpszTFq30SUWuTJMgszVnomYJ4f181JDXZ4SLgr-N0l0DvxMh1_OTkX4JBoo4h8fS319cbqaf0lhgM=w1366-h646 "Много индексов, 5% чтение, 95% запись, аггрегированные данные")

**Средний Latency: 125**

На удивление хороший результат, быть может добавил не так уж много индексов.

* [Github репозиторий](https://github.com/zzzzzzzzzzzzz/highload-test "highload-test")
* [Apache JMeter](http://jmeter.apache.org/ "JMeter")
* [Django](https://www.djangoproject.com/ "Django")
* [Django REST framework](http://www.django-rest-framework.org/ "Django REST")
* [Django Fixtures](https://code.djangoproject.com/wiki/Fixtures "Fixtures")